# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

As this assignment utilizes global variables in the skeleton code, to avoid having errors in your code you can either:

1. You can place all of your code within the function definitions for all of the questions (other than import statements).
2. You can create copies of all the global variables with the copy() method and proceed as usual.

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [314]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_nhl_df():
    # load data
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities=cities[["Metropolitan area","Population (2016 est.)[8]","NHL"]] #select only these 3 columns
#----------------------------------------------------------------------------------------------------
    # cleaning the cities dataframe  METROPOLITAN
    cities["NHL"] = cities["NHL"].apply(lambda x: re.sub(r"\[.+\]", "", x)) # removes the [] from file teams of NHL 
    cities["NHL"] = cities["NHL"].replace({"RangersIslandersDevils": "Rangers,Islanders,Devils",
                                           "KingsDucks": "Kings,Ducks"})  #These are the names of different teams so by replacing them with commas it is helpful to split afterwards                                     
    cities["NHL"] = cities["NHL"].apply(lambda x: x.split(",")) #Splits the cities["NHL"] by , operator
    cities = cities.explode("NHL") #explode(), takes the previous splited string result and prints it in different lines
    #print("Cities after explode : ",cities.shape)
    #cities.to_excel("Cities.xlsx") #gets all the entries from Metropolitan html 
#----------------------------------------------------------------------------------------------------
    # cleaning the nhl_df dataframe NHL
    nhl_df = nhl_df[nhl_df["year"] == 2018] #we want only the 2018 year
    #print(nhl_df[["team","W","L"]]) #get only 3 columns of the whole file
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.replace("*", "")) #remove * from team column
    nhl_df["team"] = nhl_df["team"].replace({"Detroit Red Wings": "Detroit Red,Wings", 
                                           "Toronto Maple Leafs": "Toronto Maple,Leafs", 
                                           "Columbus Blue Jackets": "Columbus Blue,Jackets",
                                           "Vegas Golden Knights": "Vegas Golden,Knights" })
    nhl_df["team"] = nhl_df["team"].apply(lambda x: x.split(' ')[-1]) #split the above replaced strings according to \s -> space
    nhl_df["team"] = nhl_df["team"].replace({"Red,Wings": "Red Wings", 
                                           "Maple,Leafs": "Maple Leafs", 
                                           "Blue,Jackets": "Blue Jackets",
                                           "Golden,Knights": "Golden Knights" }) #since it is not right to print the teams separated by comma, replace the 
                                                                                #commas once again with space charachter
    #print("NHL : ",nhl_df.shape)
    print("NHL+ CITIES before MERGE",nhl_df.shape)
    # merge the dataframes
    df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team") #merge the cities and dhl_df dataframes to a new one according to 
                                                                  #the columns refering to the teams
    #df.to_excel('MergedCitiesNHL_df.xlsx')
    print("NHL+ CITIES AFTER MERGE",df.shape)

    df = df[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]] #these are the columns need to print
    df["W-L%"] = df["W"].astype("int")/(df["W"].astype("int") + df["L"].astype("int"))  #calculate the W-L ratio(%) as integer type 

    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float") 
    df["W-L%"] = df["W-L%"].astype("float")#transform the population,W and L% to float number

    # drop duplicated columns
    df.loc[df["Metropolitan area"] == "New York City", "W-L%"]= 0.5182013333333334 #0.5182014205986808 # mean of NY W-L%
    #print(df.mean())
    df.loc[df["Metropolitan area"] == "Los Angeles", "W-L%"]= 0.6228945 #0.622894633764199 # mean of LA W-L%
    #print(df.mean())
    df = df.drop_duplicates(subset="Metropolitan area").reset_index()
    df = df.drop(columns="index")
    #print("NHL+ CITIES AFTER AVERAGE ",df.shape)
    return df
def nhl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    df = clean_nhl_df()

    population_by_region = df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    print(population_by_region.shape)
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    print(win_loss_by_region.shape)
    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    result = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return result[0]
nhl_correlation()

NHL+ CITIES before MERGE (35, 15)
NHL+ CITIES AFTER MERGE (31, 18)
(28,)
(28,)


0.012485959345532892

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [315]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_nba_df():
    # load data
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities=cities[["Metropolitan area","Population (2016 est.)[8]","NBA"]] #select only these 3 columns
    #print("Initial cities Dataframe with metropolitan cities: ",cities.shape) #shape (51,3)
#----------------------------------------------------------------------------------------------------
    # cleaning the cities dataframe  METROPOLITAN
    cities["NBA"] = cities["NBA"].apply(lambda x: re.sub(r"\[.+\]", "", x)) # removes the [] from file teams of NBA 
    cities["NBA"] = cities["NBA"].replace({"KnicksNets": "Knicks,Nets",
                                           "LakersClippers": "Lakers,Clippers"})  #These are the names of different teams so by replacing them with commas it is helpful to split afterwards                                     
    
    cities["NBA"] = cities["NBA"].apply(lambda x: x.split(",")) #Splits the cities["NHL"] by , operator
    #print("Cities before explode : ",cities.shape) #shape (51,3)
    cities.to_excel("CitiesNBA.xlsx") #gets all the entries from Metropolitan html 

    cities = cities.explode("NBA") #explode(), takes the previous splited string result and prints it in different lines
    #print("Cities after explode : ",cities.shape) #shape (53,3)
    cities.to_excel("CitiesNBA.xlsx") #gets all the entries from Metropolitan html 
#----------------------------------------------------------------------------------------------------
    # cleaning the nba_df dataframe NBA
    nba_df = nba_df[nba_df["year"] == 2018] #we want only the 2018 year  
    nba_df=nba_df[["team","W","L"]] #get only 3 columns of the whole file
    #print("After choosing to print only 3 columns of the file",nba_df.shape) #shape(30,3)
   
    nba_df["team"] = nba_df["team"].apply(lambda x: re.sub(r"(\*)*\s\(\d+\)", "",x)) #remove * from team column
    
    nba_df["team"] = nba_df["team"].replace({"Brooklyn Nets": "New York Nets"})
    nba_df["team"] = nba_df["team"].apply(lambda x: x.split(' ')[-1]) #split the above replaced strings according to \s -> space
    
    nba_df["team"] = nba_df["team"].replace({"Blazers": "Trail Blazers"}) #NBA_df file acording to the previous action changed Trail Blazers to Blazers 
                                                                          #and now we take it back to its initial form
    #print("NBA teams before MERGE",nba_df.shape)
    nba_df.to_excel("NBA_df.xlsx") #gets all the entries from nba.xlsx

#------------------------------------------------------------------------------------------------------------    
    # merge the dataframes
    df = pd.merge(cities, nba_df, left_on="NBA", right_on="team") #merge the cities and dhl_df dataframes to a new one according to 
                                                                  #the columns refering to the teams
    #print("NBA and Cities dataframes AFTER merge",df.shape) #shape(30, 6)

    df["W-L%"] = df["W"].astype("float")/(df["W"].astype("float") + df["L"].astype("float"))  #calculate the W-L ratio(%) as integer type 
    #print(df["W-L%"])
    df = df[["Metropolitan area", "Population (2016 est.)[8]","NBA","team", "W", "L","W-L%"]] #these are the columns need to print
    #print("File after merge and ratio calculation: ",df.shape) #shape(30, 7) includes both a column NBA (with teams) and a column teams which are the same
    #df.to_excel("NBA_teamsAfterMerge.xlsx") #gets all the entries from nba.xlsx

    # turns everything to float for the calculation of correlation
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float") 
    #df["W-L%"] = df["W-L%"].astype("float")#transform the population,W-L% to float number
    
 #-------------------------------------------------------------------------------------------------------------   
    # drop duplicated columns
    #df1=df[df["Metropolitan area"]=="New York City"] #for debugging
    #print(df1)
    #df2=df[df["Metropolitan area"]=="Los Angeles"]  #for debugging
    #print(df2)

    df = df[["Metropolitan area", "Population (2016 est.)[8]", "W-L%"]]  #shape(30,3)
    df1=df.groupby("Metropolitan area").mean()["W-L%"]   #we have 2 rows with NewYork and 2 with LA so we calculate the mean here shape(28,)
    df2 = df[["Metropolitan area", "Population (2016 est.)[8]"]].drop_duplicates()
    #print(df2.shape) # shape(28,2)

    df = pd.merge(df1, df2, left_on="Metropolitan area", right_on="Metropolitan area")
    #print(df.shape)  #shape(28,3)

    #df.to_excel("NBA_teamsAfterMerge.xlsx") #gets all the entries from nba.xlsx
    #print(df)
 #-------------------------------------------------------------------------------------------------------------   
    return df

def nba_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    df = clean_nba_df()

    population_by_region = df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    print(population_by_region.shape)
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    print(win_loss_by_region.shape)

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NBA"
    
    result = stats.pearsonr(population_by_region, win_loss_by_region)
    
    return result
nba_correlation()

(28,)
(28,)


(-0.17657160252844617, 0.36874741604463)

In [307]:
nba_correlation()

(-0.17657160252844617, 0.36874741604463)

youtube introduction to data science in python week4 assignment4 coursera
programming with danish
Ass3 (0.15052304487104848)   
Ass4 (0.004922112149349429)  0.004922112149349428

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [321]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
#print(cities)

def clean_mlb_df():
    # load data
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities=cities[["Metropolitan area","Population (2016 est.)[8]","MLB"]] #select only these 3 columns
    #print("Initial cities Dataframe with metropolitan cities: ",cities.shape) #shape (51,3)
#----------------------------------------------------------------------------------------------------
    # cleaning the cities dataframe  METROPOLITAN
    cities["MLB"] = cities["MLB"].apply(lambda x: re.sub(r"\[.+\]", "", x)) # removes the [] from file teams of MLB 
    #cities.to_excel("CitiesMLB.xlsx")

    cities["MLB"] = cities["MLB"].replace({"YankeesMets": "Yankees,Mets",
                                           "DodgersAngels": "Dodgers,Angels",
                                           "GiantsAthletics":"Giants,Athletics",
                                           "CubsWhite Sox":"Cubs,White Sox"})  #These are the names of different teams so by replacing them with commas it is helpful to split afterwards                                     
    
    cities["MLB"] = cities["MLB"].apply(lambda x: x.split(",")) #Splits the cities["MLB"] by , operator
    #print("Cities before explode : ",cities.shape) #shape (51,3)
    #cities.to_excel("CitiesMLB.xlsx") #gets all the entries from Metropolitan html 

    cities = cities.explode("MLB") #explode(), takes the previous splited string result and prints it in different lines
    #print("Cities after explode : ",cities.shape) #shape (55,3)
    cities.to_excel("CitiesMLB.xlsx") #gets all the entries from Metropolitan html 
#----------------------------------------------------------------------------------------------------
    # cleaning the nba_df dataframe NBA
    mlb_df = mlb_df[mlb_df["year"] == 2018] #we want only the 2018 year  
    mlb_df=mlb_df[["team","W","L"]] #get only 3 columns of the whole file
    #print("After choosing to print only 3 columns of the file",mlb_df.shape) #shape(30,3)
    #mlb_df.to_excel("MLB_df.xlsx") #gets all the entries from nba.xlsx
    
    mlb_df["team"] = mlb_df["team"].replace({"Oakland Athletics": "San Francisco Bay Area Athletics",
                                               "San Francisco Giants" : "San Francisco Bay Area Giants",
                                               "Boston Red Sox":"Boston RedSox",   
                                               "Chicago White Sox":"Chicago WhiteSox"})#NBA_df file acording to the previous action changed Trail Blazers to Blazers 
                                                                             #and now we take it back to its initial form 
    mlb_df["team"] = mlb_df["team"].apply(lambda x: x.split(' ')[-1]) #split the above replaced strings according to \s -> space
    mlb_df["team"] = mlb_df["team"].replace({ "RedSox":"Red Sox",
                                                "Jays": "Blue Jays",
                                                "WhiteSox":"White Sox"})  
    
    #print("MLB teams before MERGE",mlb_df.shape)  #shape(30,3)
    mlb_df.to_excel("MLB_df.xlsx") #gets all the entries from nba.xlsx

#------------------------------------------------------------------------------------------------------------    
    # merge the dataframes
    df = pd.merge(cities, mlb_df, left_on="MLB", right_on="team") #merge the cities and mlb_df dataframes to a new one according to 
                                                                  #the columns refering to the teams
    #print("MLB and Cities dataframes AFTER merge",df.shape) #shape(30, 6)

    df["W-L%"] = df["W"].astype("float")/(df["W"].astype("float") + df["L"].astype("float"))  #calculate the W-L ratio(%) as integer type 
    #print(df["W-L%"])
    df = df[["Metropolitan area", "Population (2016 est.)[8]","MLB","team", "W", "L","W-L%"]] #these are the columns need to print
    #print("File after merge and ratio calculation: ",df.shape) #shape(30, 7) includes both a column MLB (with teams) and a column teams which are the same

    # turns everything to float for the calculation of correlation
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float") 
    df.to_excel("MLB_teamsAfterMerge.xlsx") #gets all the entries from nba.xlsx

 #-------------------------------------------------------------------------------------------------------------   
    # drop duplicated columns
    #df1=df[df["Metropolitan area"]=="New York City"] #for debugging
    #print(df1)
    #df2=df[df["Metropolitan area"]=="Los Angeles"]  #for debugging
    #print(df2)

    df = df[["Metropolitan area", "Population (2016 est.)[8]", "W-L%"]]  #shape(30,3)
    #print(df.shape)
    df1=df.groupby("Metropolitan area").mean()["W-L%"]   #we have 2 rows with NewYork and 2 with LA so we calculate the mean here shape(28,)
    df2 = df[["Metropolitan area", "Population (2016 est.)[8]"]].drop_duplicates()
    #print(df2.shape) # shape(26,2)

    df = pd.merge(df1, df2, left_on="Metropolitan area", right_on="Metropolitan area")
    #print(df.shape)  #shape(26,3)

    df.to_excel("MLB_teamsAfterMerge.xlsx") #gets all the entries from nba.xlsx
    #print(df)
 #-------------------------------------------------------------------------------------------------------------   
    return df

def mlb_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    df = clean_mlb_df()
    population_by_region = df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    win_loss_by_region = df["W-L%"] 
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    result= stats.pearsonr(population_by_region, win_loss_by_region)
    
    return result[0]
mlb_correlation()

0.15027698302669307

In [ ]:
0.15052304487104848

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [318]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def clean_nfl_df():
    # load data
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities=cities[["Metropolitan area","Population (2016 est.)[8]","NFL"]] #select only these 3 columns
    #print("Initial cities Dataframe with metropolitan cities: ",cities.shape) #shape (51,3)
#----------------------------------------------------------------------------------------------------
    # cleaning the cities dataframe  METROPOLITAN
    cities["NFL"] = cities["NFL"].apply(lambda x: re.sub(r"\[.+\]", "", x)) # removes the [] from file teams of MLB 

    cities["NFL"] = cities["NFL"].replace({"GiantsJets": "Giants,Jets",
                                             "RamsChargers": "Rams,Chargers",
                                             "49ersRaiders":"49ers,Raiders"})  #These are the names of different teams so by replacing them with commas 
                                                                                 #it is helpful to split afterwards                                     
    cities.to_excel("CitiesNFL.xlsx")

    cities["NFL"] = cities["NFL"].apply(lambda x: x.split(",")) #Splits the cities["MLB"] by , operator
    #print("Cities before explode : ",cities.shape) #shape (51,3)
    #cities.to_excel("CitiesNFL.xlsx") #gets all the entries from Metropolitan html 

    cities = cities.explode("NFL") #explode(), takes the previous splited string result and prints it in different lines
    #print("Cities after explode : ",cities.shape) #shape (54,3)
    cities.to_excel("CitiesNFL.xlsx") #gets all the entries from Metropolitan html 
#----------------------------------------------------------------------------------------------------
    # cleaning the nba_df dataframe NBA
    nfl_df = nfl_df[nfl_df["year"] == 2018] #we want only the 2018 year  
    nfl_df=nfl_df[["team","W","L"]] #get only 3 columns of the whole file
    #print("After choosing to print only 3 columns of the file",nfl_df.shape) #shape(40,3)
    nfl_df["team"] = nfl_df["team"].apply(lambda x: re.sub(r"(\*|\+)", "",x)) #remove * from team column
    #nfl_df.to_excel("NFL_df.xlsx") #gets all the entries from nba.xlsx

    nfl_df["team"] = nfl_df["team"].replace({"New York Jets":"New York City Jets",
                                                "New York Giants":"New York City Giants",
                                                "Oakland Raiders": "San Francisco Bay Area Raiders",
                                               "San Francisco 49ers" : "San Francisco Bay Area 49ers",
                                               "Dallas Cowboys":"Dallas-Fort Cowboys",   
                                               "Washington Redskins":"Washington,D.C. Redskins",
                                               "New England Patriots":"Boston Patriots",
                                               "Minnesota Vikings":"Minneapolis-Saint Paul Vikings",
                                               "Miami Dolphins":"Miami–Fort Lauderdale Dolphins",
                                               "Arizona Cardinals":"Phoenix Cardinals",
                                               "Tampa Bay Buccaneers":"Tampa Bay Area Buccaneers",
                                               "Carolina Panthers":"Charlotte Panthers",
                                               "Tennessee Titans":"Nashville Titans"})  

    nfl_df["team"] = nfl_df["team"].apply(lambda x: x.split(' ')[-1]) #split the above replaced strings according to \s -> space
    
    #print("NFL teams before MERGE",nfl_df.shape)  #shape(40,3)
    #nfl_df.to_excel("NFL_df.xlsx") #gets all the entries from nba.xlsx
#------------------------------------------------------------------------------------------------------------    
       # merge the dataframes
    df = pd.merge(cities, nfl_df, left_on="NFL", right_on="team") #merge the cities and mlb_df dataframes to a new one according to 
                                                                  #the columns refering to the teams
    #print("NFL and Cities dataframes AFTER merge",df.shape) #shape(30, 6)
    #df.to_excel("NFL_teamsAfterMerge.xlsx")

    # MEAN Calculation
    df = df[["Metropolitan area", "Population (2016 est.)[8]","W","L"]]  #shape(32,3)
    #print(df)
    df["W"]=df["W"].astype("float")
    df["L"]=df["L"].astype("float")
    #print(df)
    df=df.groupby(["Metropolitan area","Population (2016 est.)[8]"])[["W","L"]].mean() # creates all the columns needed and calculates the mean and drops duplicates
    #print("df   ",df.shape) #shape(29,2)
    df["W-L%"] = df["W"]/(df["W"]+ df["L"])  #adds the W-L% column
    #print(df.index)
    df=df.reset_index(level=1)
    df["Population (2016 est.)[8]"] = df["Population (2016 est.)[8]"].astype("float")
    #print(df.shape)

    #print(df["Population (2016 est.)[8]"].shape)
    df.to_excel("NFL_teamsAfterMerge.xlsx") #gets all the entries from nfl.xlsx
    #print("File after merge and ratio calculation: ",df.shape) #shape(29, 3) includes both a column MLB (with teams) and a column teams which are the same
 #-------------------------------------------------------------------------------------------------------------   
    return df
def nfl_correlation(): 
    # YOUR CODE HERE
    #raise NotImplementedError()
    df = clean_nfl_df()

    population_by_region = df["Population (2016 est.)[8]"] # pass in metropolitan area population from cities
    print(population_by_region.shape)
    win_loss_by_region = df["W-L%"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    print(win_loss_by_region.shape)
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"
    result= stats.pearsonr(population_by_region, win_loss_by_region)
    
    return result[0]
nfl_correlation()

(29,)
(29,)


0.004922112149349428

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values